In [17]:
%load_ext autoreload
%autoreload 2

from utils.hamiltonians import Max3satHamiltonian
from utils.qaoa import QAOA
from qiskit import transpile
from qiskit.circuit import QuantumCircuit
from utils.circuit_utils import calculate_expected_fidelity, calculate_expected_fidelity_ideal, calculate_swap_overhead
from utils.experiments import SuperconductingExperiment
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, EstimatorV2 as Estimator
import numpy as np
import pandas as pd

hamiltonian = Max3satHamiltonian("./instances/uf20-01.cnf")

print(hamiltonian.formula.nv, hamiltonian.formula.clauses)
print(hamiltonian.get_pauli_list(), len(hamiltonian.get_pauli_list()))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
20 [[4, -18, 19], [3, 18, -5], [-5, -8, -15], [-20, 7, -16], [10, -13, -7], [-12, -9, 17], [17, 19, 5], [-16, 9, 15], [11, -5, -14], [18, -10, 13], [-3, 11, 12], [-6, -17, -8], [-18, 14, 1], [-19, -15, 10], [12, 18, -19], [-8, 4, 7], [-8, -9, 4], [7, 17, -15], [12, -7, -14], [-10, -11, 8], [2, -15, -11], [9, 6, 1], [-11, 20, -17], [9, -15, 13], [12, -7, -17], [-18, -2, 20], [20, 12, 4], [19, 11, 14], [-16, 18, -4], [-1, -17, -19], [-13, 15, 10], [-12, -14, -13], [12, -14, -7], [-7, 16, 10], [6, 10, 7], [20, 14, -16], [-19, 17, 11], [-7, 1, -20], [-5, 12, 15], [-4, -9, -13], [12, -11, -7], [-5, 19, -8], [1, 16, 17], [20, -14, -15], [13, -4, 10], [14, 7, 10], [-5, 9, 20], [10, 1, -19], [-16, -15, -1], [16, 3, -11], [-15, -10, 4], [4, -15, -3], [-10, -16, 11], [-8, 12, -5], [14, -6, 12], [1, 6, 11], [-13, -5, -1], [-7, -2, 12], [1, -20, 19], [-2, -13, -8], [15, 18, 4], [-11, 14, 9], [-6, -15, -2], [5, 

In [18]:
basis_gates = ["rx", "rz", "x", "y", "z", "h", "id", "cz"]
depth = 1
qaoa = QAOA(hamiltonian)
qaoa_circuit, cost_params, mixer_params = qaoa.naive_qaoa_circuit(depth)
bound_circuit = qaoa_circuit.assign_parameters({cost_params: [np.pi / 2.123 for param in cost_params], mixer_params: [np.pi / 3.123 for param in mixer_params]})
bound_circuit.measure_all()
transpiled_circuit = transpile(bound_circuit, basis_gates=basis_gates, optimization_level=3)

In [19]:
service = QiskitRuntimeService()

In [20]:
backend = service.backend("ibm_sherbrooke")

In [21]:
print(backend.basis_gates)

['ecr', 'id', 'rz', 'sx', 'x']


In [5]:
eagle_circuit = transpile(bound_circuit, backend=backend, optimization_level=3)

print(eagle_circuit.depth(), bound_circuit.depth())
print(eagle_circuit.count_ops(), bound_circuit.count_ops())

2585 191
OrderedDict([('rz', 4466), ('sx', 2911), ('ecr', 1281), ('x', 364), ('measure', 20), ('barrier', 1)]) OrderedDict([('cx', 295), ('rz', 231), ('rx', 20), ('measure', 20), ('barrier', 1)])


In [23]:
ost_operator = hamiltonian.get_sparse_pauli_operator()
observable = cost_operator.apply_layout(eagle_circuit.layout)
estimator = Estimator(backend=backend)

In [25]:
job = estimator.run([(eagle_circuit, observable)])

In [29]:
print(f"Job ID: {job.job_id()}")
print(f"Job Status: {job.status()}")

Job ID: crx7kt77wv80008fkzm0
Job Status: QUEUED


In [31]:
least_busy_backend = service.least_busy(operational=True, simulator=False)

print(least_busy_backend)

<IBMBackend('ibm_osaka')>


In [46]:
sampler = Sampler(backend=backend)
estimator = Estimator(backend=backend)
eagle_circuit = transpile(bound_circuit, backend=backend, optimization_level=3)
observable = cost_operator.apply_layout(eagle_circuit.layout)

print(eagle_circuit.depth(), bound_circuit.depth())
print(eagle_circuit.count_ops(), bound_circuit.count_ops())

2539 191
OrderedDict([('rz', 4546), ('sx', 2892), ('ecr', 1288), ('x', 405), ('measure', 20), ('barrier', 1)]) OrderedDict([('cx', 295), ('rz', 231), ('rx', 20), ('measure', 20), ('barrier', 1)])


In [49]:
sampler_job = sampler.run([eagle_circuit], shots=256)

In [43]:
job = estimator.run([(eagle_circuit, observable)])

/home/lax/Desktop/FPQA-MAX3-SAT/.venv/lib/python3.10/site-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:879: UserWarning: Your current pending jobs are estimated to consume 623.1034379365619 quantum seconds, but you only have 600 quantum seconds left in your monthly quota; therefore, it is likely this job will be canceled
  warnings.warn(warning_message)


In [7]:
print(calculate_expected_fidelity(eagle_circuit, backend))
print(calculate_swap_overhead(transpiled_circuit, eagle_circuit))

(7.195686173181647e-12, 0.0001808796796848703, 152648674309, 6073)
328


In [31]:
experiment = SuperconductingExperiment("superconducting_1", ["uf20-01.cnf", "uuf50-02.cnf"], backend)

experiment.run(max_depth=1, max_iterations=2)

>>> Starting experiments with Problem: uf20-01.cnf ...
>>> Parametrized circuit created, Depth: 1 ...
>>> Transpiling parametrized circuit, Iteration: 0 ...
>>> Transpiling parametrized circuit, Iteration: 1 ...
>>> Starting experiments with Problem: uuf50-02.cnf ...
>>> Parametrized circuit created, Depth: 1 ...
>>> Transpiling parametrized circuit, Iteration: 0 ...


/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:34: RuntimeWarning: divide by zero encountered in scalar divide
  estimated_shots_without_decoherence = np.log(1 - DESIRED_SUCCESS_PROBABILITY) / np.log(1 - fidelity)
/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:35: RuntimeWarning: divide by zero encountered in scalar divide
  estimated_shots = np.log(1 - DESIRED_SUCCESS_PROBABILITY) / np.log(1 - fidelity * decoherence_fidelity)
/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:48: RuntimeWarning: divide by zero encountered in scalar divide
  estimated_shots = np.log(1 - DESIRED_SUCCESS_PROBABILITY) / np.log(1 - fidelity)


>>> Transpiling parametrized circuit, Iteration: 1 ...


/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:34: RuntimeWarning: divide by zero encountered in scalar divide
  estimated_shots_without_decoherence = np.log(1 - DESIRED_SUCCESS_PROBABILITY) / np.log(1 - fidelity)
/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:35: RuntimeWarning: divide by zero encountered in scalar divide
  estimated_shots = np.log(1 - DESIRED_SUCCESS_PROBABILITY) / np.log(1 - fidelity * decoherence_fidelity)
/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:48: RuntimeWarning: divide by zero encountered in scalar divide
  estimated_shots = np.log(1 - DESIRED_SUCCESS_PROBABILITY) / np.log(1 - fidelity)


,Name,Hardware,QAOA Depth,Average Circuit Depth,Ideal Fidelity,Ideal Estimated Shots,Fidelity with Decoherence,Estimated Shots with Decoherence for 2/3 Success Rate,Fidelity without Decoherence,Estimated Shots without Decoherence for 2/3 Success Rate,Average Swap Overhead,Average Number of Two Qubit Gates,Iterations
0,uf20-01.cnf,ibm_sherbrooke,1,2437.5,6.437717e-07,1.843287e+06,1.072906e-11,4.070923e+11,0.000142,1.825479e+04,331,1287,2
1,uuf50-02.cnf,ibm_sherbrooke,1,9413.5,4.113517e-33,-inf,0.000000e+00,-inf,0.000000,-inf,1942,6708,2


In [74]:
from utils.fake_backend import create_fake_heavy_hex_backend

fake_backend = create_fake_heavy_hex_backend(8, 4)

print(fake_backend.num_qubits)
print(fake_backend.coupling_map)
print(fake_backend.target["ecr"][(0,1)])
print(fake_backend.basis_gates)

207
[[0, 1], [0, 17], [17, 22], [1, 2], [2, 3], [3, 4], [4, 5], [4, 18], [18, 26], [5, 6], [6, 7], [7, 8], [8, 9], [8, 19], [19, 30], [9, 10], [10, 11], [11, 12], [12, 13], [12, 20], [20, 34], [13, 14], [14, 15], [15, 16], [16, 21], [21, 38], [22, 23], [23, 24], [24, 25], [24, 41], [41, 48], [25, 26], [26, 27], [27, 28], [28, 29], [28, 42], [42, 52], [29, 30], [30, 31], [31, 32], [32, 33], [32, 43], [43, 56], [33, 34], [34, 35], [35, 36], [36, 37], [36, 44], [44, 60], [37, 38], [38, 39], [39, 40], [40, 45], [45, 64], [46, 47], [46, 65], [65, 70], [47, 48], [48, 49], [49, 50], [50, 51], [50, 66], [66, 74], [51, 52], [52, 53], [53, 54], [54, 55], [54, 67], [67, 78], [55, 56], [56, 57], [57, 58], [58, 59], [58, 68], [68, 82], [59, 60], [60, 61], [61, 62], [62, 63], [62, 69], [69, 86], [63, 64], [70, 71], [71, 72], [72, 73], [72, 89], [89, 96], [73, 74], [74, 75], [75, 76], [76, 77], [76, 90], [90, 100], [77, 78], [78, 79], [79, 80], [80, 81], [80, 91], [91, 104], [81, 82], [82, 83], [83, 